***Inference Pipeline***

In [19]:
import pandas as pd
import json
import os
from dotenv import load_dotenv
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.llms import HuggingFaceHub
from langchain_community.vectorstores import Neo4jVector
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
load_dotenv()

def graphsearch(question):
    # Gemini (https://aistudio.google.com/app/apikey)
    gemini_api = os.getenv("GEMINI_API")

    # Hugging Face (if we want to use open source LLM)
    hf_api = os.getenv("HF_API")

    # Neo4j 
    neo4j_url = os.getenv("NEO4J_CONNECTION_URL")
    neo4j_user = os.getenv("NEO4J_USER")
    neo4j_password = os.getenv("NEO4J_PASSWORD")

    # https://api.python.langchain.com/en/latest/graphs/langchain_community.graphs.neo4j_graph.Neo4jGraph.html
    graph = Neo4jGraph(neo4j_url,neo4j_user,neo4j_password)


    examples= [
        {
            "question": "Which workers speak French?",
            "query": "MATCH (p:Person)-[:SPEAKS]->(l:Language {{name: 'French'}}) RETURN p.name",
        },
        {
            "question": "What industries are workers named Emily associated with?",
            "query": "MATCH (p:Person {{name: 'Emily'}})-[:WORKS_IN]->(c:Company)-[:IS_IN]->(i:Industry) RETURN i.name",
        },
        {
            "question": "Which workers live in Canada and speak German?",
            "query": "MATCH (p:Person)-[:LIVES_IN]->(:Country {{name: 'Canada'}}), (p)-[:SPEAKS]->(:Language {{name: 'German'}}) RETURN p.name",
        },
        {
            "question": "In which countries do workers who speak Spanish live?",
            "query": "MATCH (p:Person)-[:SPEAKS]->(:Language {{name: 'Spanish'}})<-[:SPEAKS]-(worker:Person)-[:LIVES_IN]->(c:Country) RETURN DISTINCT c.name AS Country",
        },
        {
            "question": "What companies do workers named John work in?",
            "query": "MATCH (p:Person {{name: 'John'}})-[:WORKS_IN]->(c:Company) RETURN c.name",
        },
        {
            "question":"How many workers in Hospital and Health Care industry able to speak Korea",
            "query": "MATCH (p:Person)-[:WORKS_IN]->(:Company)-[:IS_IN]->(:Industry {{name: 'Hospitals and Health Care'}}),(p)-[:SPEAKS]->(:Language {{name: 'Korean'}}) RETURN COUNT(DISTINCT p) AS NumberOfWorkers",
        },
        {
            "question": "What companies are located in the technology industry?",
            "query": "MATCH (c:Company)-[:IS_IN]->(:Industry {{name: 'Technology'}}) RETURN c.name",
        },
        {
            "question": "Where do workers named Alice live?",
            "query": "MATCH (p:Person {{name: 'Alice'}})-[:LIVES_IN]->(c:Country) RETURN c.name",
        },
    ]

    llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key = gemini_api ,temperature = 0)

    example_prompt = PromptTemplate.from_template(
        "User input: {question}\nCypher query: {query}"
    )

    example_selector = SemanticSimilarityExampleSelector.from_examples(
        examples,
        HuggingFaceEmbeddings(),
        Neo4jVector,
        url = neo4j_url,
        username = neo4j_user,
        password = neo4j_password,
        k=3,
        input_keys=["question"],
    )

    example_selector.select_examples({"question": f'{question}'})

    dynamic_prompt = FewShotPromptTemplate(
        example_selector=example_selector,
        example_prompt=example_prompt,
        prefix="You are a Neo4j expert. Given an input question, create a syntactically correct Cypher query to run.\n\nHere is the schema information\n{schema}.\n\nBelow are a number of examples of questions and their corresponding Cypher queries.",
        suffix="User input: {question}\nCypher query: ",
        input_variables=["question", "schema"],
    )

    chain2 = GraphCypherQAChain.from_llm(graph=graph, llm=llm, cypher_prompt=dynamic_prompt, verbose=True)

    print(dynamic_prompt.format(question=question, schema="foo"))

    response = chain2.invoke(question)['result']
    print(response)

In [20]:
graphsearch('Where is Paul Lukes working?')

You are a Neo4j expert. Given an input question, create a syntactically correct Cypher query to run.

Here is the schema information
foo.

Below are a number of examples of questions and their corresponding Cypher queries.

User input: What companies do workers named John work in?
Cypher query: MATCH (p:Person {name: 'John'})-[:WORKS_IN]->(c:Company) RETURN c.name

User input: Where do workers named Alice live?
Cypher query: MATCH (p:Person {name: 'Alice'})-[:LIVES_IN]->(c:Country) RETURN c.name

User input: What industries are workers named Emily associated with?
Cypher query: MATCH (p:Person {name: 'Emily'})-[:WORKS_IN]->(c:Company)-[:IS_IN]->(i:Industry) RETURN i.name

User input: Where is Paul Lukes working?
Cypher query: 


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: 'Paul Lukes'})-[:WORKS_IN]->(c:Company) RETURN c.name
Full Context:
[{'c.name': 'Toolbox Creative'}]

> Finished chain.
I don't know the answer.


In [2]:
import pandas as pd
import json
import os
from dotenv import load_dotenv
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.llms import HuggingFaceHub
from langchain_community.vectorstores import Neo4jVector
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
load_dotenv()

/Users/chris/Projects/KnowledgeGraph/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/chris/Projects/KnowledgeGraph/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [4]:

# Gemini (https://aistudio.google.com/app/apikey)
gemini_api = os.getenv("GEMINI_API")

# Hugging Face (if we want to use open source LLM)
hf_api = os.getenv("HF_API")

# Neo4j 
neo4j_url = os.getenv("NEO4J_CONNECTION_URL")
print(neo4j_url)
neo4j_user = os.getenv("NEO4J_USER")
print(neo4j_user)
neo4j_password = os.getenv("NEO4J_PASSWORD")

# https://api.python.langchain.com/en/latest/graphs/langchain_community.graphs.neo4j_graph.Neo4jGraph.html
graph = Neo4jGraph(neo4j_url,neo4j_user,neo4j_password)


neo4j+s://19bd88a6.databases.neo4j.io:7687
neo4j


In [5]:
examples= [
    {
        "question": "Which workers speak French?",
        "query": "MATCH (p:Person)-[:SPEAKS]->(l:Language {{name: 'French'}}) RETURN p.name",
    },
    {
        "question": "What industries are workers named Emily associated with?",
        "query": "MATCH (p:Person {{name: 'Emily'}})-[:WORKS_IN]->(c:Company)-[:IS_IN]->(i:Industry) RETURN i.name",
    },
    {
        "question": "Which workers live in Canada and speak German?",
        "query": "MATCH (p:Person)-[:LIVES_IN]->(:Country {{name: 'Canada'}}), (p)-[:SPEAKS]->(:Language {{name: 'German'}}) RETURN p.name",
    },
    {
        "question": "In which countries do workers who speak Spanish live?",
        "query": "MATCH (p:Person)-[:SPEAKS]->(:Language {{name: 'Spanish'}})<-[:SPEAKS]-(worker:Person)-[:LIVES_IN]->(c:Country) RETURN DISTINCT c.name AS Country",
    },
    {
        "question": "What companies do workers named John work in?",
        "query": "MATCH (p:Person {{name: 'John'}})-[:WORKS_IN]->(c:Company) RETURN c.name",
    },
    {
        "question":"How many workers in Hospital and Health Care industry able to speak Korea",
        "query": "MATCH (p:Person)-[:WORKS_IN]->(:Company)-[:IS_IN]->(:Industry {{name: 'Hospitals and Health Care'}}),(p)-[:SPEAKS]->(:Language {{name: 'Korean'}}) RETURN COUNT(DISTINCT p) AS NumberOfWorkers",
    },
    {
        "question": "What companies are located in the technology industry?",
        "query": "MATCH (c:Company)-[:IS_IN]->(:Industry {{name: 'Technology'}}) RETURN c.name",
    },
    {
        "question": "Where do workers named Alice live?",
        "query": "MATCH (p:Person {{name: 'Alice'}})-[:LIVES_IN]->(c:Country) RETURN c.name",
    },
]

In [6]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key = gemini_api ,temperature = 0)


In [7]:
example_prompt = PromptTemplate.from_template(
    "User input: {question}\nCypher query: {query}"
)


In [8]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    HuggingFaceEmbeddings(),
    Neo4jVector,
    url = neo4j_url,
    username = neo4j_user,
    password = neo4j_password,
    k=3,
    input_keys=["question"],
)


In [9]:
example_selector.select_examples({"question": "Where do Michael work?"})


[{'query': "MATCH (p:Person {{name: 'John'}})-[:WORKS_IN]->(c:Company) RETURN c.name",
  'question': 'What companies do workers named John work in?'},
 {'query': "MATCH (p:Person {{name: 'Alice'}})-[:LIVES_IN]->(c:Country) RETURN c.name",
  'question': 'Where do workers named Alice live?'},
 {'query': "MATCH (p:Person {{name: 'Emily'}})-[:WORKS_IN]->(c:Company)-[:IS_IN]->(i:Industry) RETURN i.name",
  'question': 'What industries are workers named Emily associated with?'}]

In [10]:
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="You are a Neo4j expert. Given an input question, create a syntactically correct Cypher query to run.\n\nHere is the schema information\n{schema}.\n\nBelow are a number of examples of questions and their corresponding Cypher queries.",
    suffix="User input: {question}\nCypher query: ",
    input_variables=["question", "schema"],
)

In [11]:
chain2 = GraphCypherQAChain.from_llm(graph=graph, llm=llm, cypher_prompt=dynamic_prompt, verbose=True)


In [12]:
questions = ["List all companies in advertising services industry!",
             "A worker who graduated from Simon Fraser University is currently employed at?",
             "Where is Paul Lukes working?",
             "A worker residing in Canada who is proficient in Vietnamese?",
             "How many worker in United States speak Urdu?"]

In [13]:
for q in questions:
    print('====== START ======')
    print(chain2.invoke(q)['result'])
    print('====== END ====== \n')

====== START ======


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Company)-[:IS_IN]->(i:Industry {name: 'Advertising Services'}) RETURN c.name
Full Context:
[{'c.name': 'Toolbox Creative'}, {'c.name': 'Baked Advertising'}, {'c.name': 'Search Engine People'}]

> Finished chain.
Toolbox Creative, Baked Advertising, Search Engine People
====== END ====== 

====== START ======


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:EDUCATED_AT]->(s:School {name: 'Simon Fraser University'})-[:WORKS_IN]->(c:Company) RETURN c.name
Full Context:
[]

> Finished chain.
I do not have that information.
====== END ====== 

====== START ======


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: 'Paul Lukes'})-[:WORKS_IN]->(c:Company) RETURN c.name
Full Context:
[{'c.name': 'Toolbox Creative'}]

> Finished chain.
I don't know the answer.
====== END ====== 

====== START ======


> Entering new GraphCypherQAChain cha

In [14]:
print(dynamic_prompt.format(question="Where do Michael work?", schema="foo"))


You are a Neo4j expert. Given an input question, create a syntactically correct Cypher query to run.

Here is the schema information
foo.

Below are a number of examples of questions and their corresponding Cypher queries.

User input: What companies do workers named John work in?
Cypher query: MATCH (p:Person {name: 'John'})-[:WORKS_IN]->(c:Company) RETURN c.name

User input: Where do workers named Alice live?
Cypher query: MATCH (p:Person {name: 'Alice'})-[:LIVES_IN]->(c:Country) RETURN c.name

User input: What industries are workers named Emily associated with?
Cypher query: MATCH (p:Person {name: 'Emily'})-[:WORKS_IN]->(c:Company)-[:IS_IN]->(i:Industry) RETURN i.name

User input: Where do Michael work?
Cypher query: 
